## Import required library


In [1]:
import sys
sys.path.append('C:/Users/sudha/anaconda3/lib/site-packages')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.models import load_model

## Read Dataset and do preprocessing

In [2]:
df = pd.read_csv("spam.csv",encoding='ISO-8859-1')

In [3]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis = 1,inplace = True)
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
df.groupby(['v1']).size()


v1
ham     4825
spam     747
dtype: int64

In [6]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [8]:
max_words = 1000
max_len = 150

In [9]:
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)


In [10]:
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


In [11]:
sequences_matrix

array([[  0,   0,   0, ...,   4,  16, 353],
       [  0,   0,   0, ..., 632,   2,   7],
       [  0,   0,   0, ...,  95,  70, 102],
       ...,
       [  0,   0,   0, ...,  12, 194, 173],
       [  0,   0,   0, ..., 165,  14, 180],
       [  0,   0,   0, ...,  75, 960,   2]])

## Create Model and Add Layers (LSTM, Dense-(Hidden Layers), Output)

In [13]:
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

## Compile the model

In [14]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

## Fit the Model

In [15]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)


Epoch 1/10
30/30 [==============================] - 16s 316ms/step - loss: 0.3178 - accuracy: 0.8820 - val_loss: 0.1318 - val_accuracy: 0.9789
Epoch 2/10
30/30 [==============================] - 9s 289ms/step - loss: 0.0835 - accuracy: 0.9802 - val_loss: 0.0352 - val_accuracy: 0.9905
Epoch 3/10
30/30 [==============================] - 9s 288ms/step - loss: 0.0479 - accuracy: 0.9860 - val_loss: 0.0335 - val_accuracy: 0.9884
Epoch 4/10
30/30 [==============================] - 10s 327ms/step - loss: 0.0387 - accuracy: 0.9908 - val_loss: 0.0344 - val_accuracy: 0.9873
Epoch 5/10
30/30 [==============================] - 10s 322ms/step - loss: 0.0323 - accuracy: 0.9902 - val_loss: 0.0347 - val_accuracy: 0.9873
Epoch 6/10
30/30 [==============================] - 12s 386ms/step - loss: 0.0227 - accuracy: 0.9939 - val_loss: 0.0346 - val_accuracy: 0.9863
Epoch 7/10
30/30 [==============================] - 11s 377ms/step - loss: 0.0196 - accuracy: 0.9947 - val_loss: 0.0467 - val_accuracy: 0.9821
E

## Save the Model

In [17]:
model.save('spam_model')

INFO:tensorflow:Assets written to: spam_model\assets


INFO:tensorflow:Assets written to: spam_model\assets


## Test the model

In [18]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))


27/27 [==============================] - 1s 36ms/step - loss: 0.0929 - accuracy: 0.9833
Accuracy: 0.983


In [19]:
y_pred = model.predict(test_sequences_matrix)
pred = y_pred[25:40].round(3)
test =Y_test[25:40]

In [22]:
print(pred)
print(test)

[[0.   ]
 [0.   ]
 [0.008]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]]
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
